# Model Context Protocol (MCP)

This notebook is a beginner-friendly, practical walkthrough of the **Model Context Protocol (MCP)**. MCP is an emerging standard that helps language-model agents share context with tools, other agents, and users in a predictable way. If you are new to agent systems, do not worry—by the end of this notebook you will understand the basic ideas and have working code examples that you can adapt for your own projects.

> **What you will learn:**
> * Why context sharing matters when building AI assistants.
> * The key concepts that MCP introduces.
> * How to structure MCP messages and validate them in Python.
> * How to simulate a small MCP interaction that fetches resources safely.

## How to use this notebook

1. Work through the sections in order. Each section builds on the previous one and introduces a single new idea.
2. Read the explanations in the markdown cells first, then run the accompanying code cells.
3. If you are brand new to Python, simply press `Shift + Enter` in each code cell to execute it. The cell outputs will appear right below the cell.
4. When you see an **Exercise**, pause and think through the guiding question before looking at the provided answer or explanation.

## Learning objectives

By the end of this notebook you should be able to:

- Explain what "context" means for an AI agent and why it must be standardized.
- Describe the main building blocks of the Model Context Protocol: sessions, resources, and messages.
- Read and write simple MCP-compliant messages in JSON.
- Run and modify Python helper functions that validate and format MCP traffic.
- Identify common safety and observability practices when exchanging context.

## Key vocabulary

| Term | Friendly definition |
| --- | --- |
| **Agent** | A program (often powered by a language model) that can reason, plan, and call tools on your behalf. |
| **Context** | All of the information the agent can currently "see": the user request, previous steps, tool results, and constraints. |
| **Protocol** | A shared agreement on message formats and behavior so that independent components can cooperate. |
| **Session** | A temporary container that groups the steps for a single user request or job. |
| **Resource** | A structured piece of data that can be shared via MCP (files, database rows, embeddings, etc.). |
| **Namespace** | A label that keeps related resources grouped together and prevents accidental cross-talk between projects. |

## Why do we need a model context protocol?

Imagine an AI assistant that can search documents, call APIs, and email summaries. Each tool speaks its own language. Without a protocol, the agent might:

- Forget to tell the tool which project the request belongs to, leaking data across teams.
- Request too many results and overload a service.
- Lose track of how to resume a multi-page response when pagination is required.

A **model context protocol** solves these issues by enforcing a predictable structure for every message:

1. The agent describes **what** it wants (for example, "Give me the latest incidents for project *alpha*").
2. It includes **metadata** such as the session identifier and continuation tokens for pagination.
3. The receiving tool validates the request before executing it and returns structured results.

The rest of this notebook explores how MCP encodes that structure.

## MCP building blocks at a glance

The MCP specification is still evolving, but most implementations rely on a few universal pieces:

1. **Session Handshake** – establishes who is speaking and what capabilities they have.
2. **Context Messages** – JSON documents that describe requests (`resource_request`, `tool_call`, etc.) and responses (`resource_response`, `error`).
3. **Reference Frames** – metadata bundles that provide the shared frame of reference for an interaction: the active namespace, available resources, and pagination hints.
4. **Guardrails** – limits such as maximum page size, allowed namespaces, and rate limits that keep the system safe.

We will create small Python helpers that encode these ideas explicitly.

In [ ]:
from dataclasses import dataclass, field, asdict
from typing import Any, Dict, List, Literal, Optional
from pprint import pprint
import json

MessageType = Literal[
    "session_init",
    "resource_request",
    "resource_response",
    "tool_call",
    "tool_result",
    "error",
]

@dataclass
class ReferenceFrame:
    """Minimal description of a reference frame in MCP."""

    session_id: str
    namespace: str
    continuation_token: Optional[str] = None
    resource_handles: List[str] = field(default_factory=list)

@dataclass
class MCPMessage:
    """A convenience wrapper for MCP messages.

    In practice the MCP spec uses JSON objects. We use a dataclass here to
    make it easier to construct and validate messages in Python.
    """

    type: MessageType
    frame: ReferenceFrame
    payload: Dict[str, Any] = field(default_factory=dict)
    constraints: Dict[str, Any] = field(default_factory=dict)

    def to_json(self) -> str:
        """Serialize the message to a JSON string."""
        return json.dumps(asdict(self), indent=2, sort_keys=True)

    def validate(self) -> None:
        """Perform simple validation checks and raise ValueError if invalid."""
        if not self.frame.session_id:
            raise ValueError("session_id must not be empty")
        if not self.frame.namespace:
            raise ValueError("namespace must not be empty")
        if self.type == "resource_request" and "resource" not in self.payload:
            raise ValueError("resource_request payload must include a 'resource' field")
        if self.type == "resource_response" and "items" not in self.payload:
            raise ValueError("resource_response payload must include an 'items' list")

        limit = self.constraints.get("limit")
        if limit is not None and (not isinstance(limit, int) or limit <= 0):
            raise ValueError("constraints.limit must be a positive integer")
        if limit is not None and limit > 100:
            raise ValueError("constraints.limit is too high; cap at 100 per MCP best practice")

    def describe(self) -> None:
        """Print a human-readable summary for teaching purposes."""
        print(f"Message type: {self.type}")
        print(f"Session: {self.frame.session_id} | Namespace: {self.frame.namespace}")
        if self.frame.continuation_token:
            print(f"Continuation token: {self.frame.continuation_token}")
        if self.frame.resource_handles:
            print(f"Resource handles: {', '.join(self.frame.resource_handles)}")
        if self.constraints:
            print(f"Constraints: {self.constraints}")
        if self.payload:
            print("Payload content:")
            pprint(self.payload)

def make_reference_frame(namespace: str, session_id: str = "session-001") -> ReferenceFrame:
    return ReferenceFrame(session_id=session_id, namespace=namespace)


### Step 1 – Constructing a session handshake

Every interaction begins with a handshake that confirms the session and negotiates capabilities. In this simplified example we describe:

- Which namespace the agent plans to access.
- Which resources it already has handles for (maybe from previous steps).
- Optional continuation tokens if the session is resuming a longer exchange.

The `describe()` helper prints the message in plain English, while `to_json()` shows the exact payload that would be sent over the wire.

In [ ]:
handshake = MCPMessage(
    type="session_init",
    frame=make_reference_frame(namespace="project-alpha"),
    payload={
        "agent_capabilities": ["vector_search", "incident_reporting"],
        "requested_tools": ["vector_store"],
    },
)

handshake.validate()
handshake.describe()
print("\nRaw JSON message:\n", handshake.to_json())


### Step 2 – Requesting a resource with guardrails

After the handshake, the agent can issue a `resource_request`. MCP encourages explicit constraints so that tools can enforce safety limits. Common constraints include:

- `limit`: the maximum number of items to return per page.
- `filter`: key-value pairs the tool should use when narrowing results.
- `sort`: instructions for ordering the response.

The example below asks the `vector_store` resource for the five most recent incident reports within the `project-alpha` namespace.

In [ ]:
resource_request = MCPMessage(
    type="resource_request",
    frame=make_reference_frame(namespace="project-alpha"),
    payload={
        "resource": "vector_store",
        "query": "recent incidents involving authentication",
    },
    constraints={
        "limit": 5,
        "sort": {"field": "timestamp", "order": "desc"},
    },
)

resource_request.validate()
resource_request.describe()
print("\nRaw JSON message:\n", resource_request.to_json())


### Step 3 – Simulating a tool response

To keep the notebook self-contained, we simulate a tool that receives the request and returns a matching `resource_response`. The response includes:

- The same reference frame (session + namespace) so the agent can match replies.
- A list of `items`, each with structured content.
- A `continuation_token` when more pages are available.

The helper function `simulate_vector_store` illustrates how a tool might apply the incoming constraints. In a real MCP deployment the tool would perform a database query or API call instead of returning static data.

In [ ]:
from typing import Iterable

def simulate_vector_store(request: MCPMessage) -> MCPMessage:
    """Return a fake response to demonstrate MCP-compatible behavior."""
    request.validate()
    fake_items = [
        {
            "id": "incident-001",
            "summary": "Authentication delays after rate limit misconfiguration.",
            "timestamp": "2024-03-03T12:30:00Z",
        },
        {
            "id": "incident-002",
            "summary": "OAuth callback timeout affecting new sign-ups.",
            "timestamp": "2024-02-27T09:15:00Z",
        },
        {
            "id": "incident-003",
            "summary": "Temporary credential leak detected and revoked.",
            "timestamp": "2024-02-10T16:45:00Z",
        },
    ]

    limit = request.constraints.get("limit", len(fake_items))
    limited_items = fake_items[:limit]

    continuation_token = None
    if limit < len(fake_items):
        continuation_token = "next-page-token"

    response_frame = ReferenceFrame(
        session_id=request.frame.session_id,
        namespace=request.frame.namespace,
        continuation_token=continuation_token,
        resource_handles=[item["id"] for item in limited_items],
    )

    response = MCPMessage(
        type="resource_response",
        frame=response_frame,
        payload={"items": limited_items},
        constraints={"limit": limit},
    )
    response.validate()
    return response

resource_response = simulate_vector_store(resource_request)
resource_response.describe()
print("\nRaw JSON message:\n", resource_response.to_json())


### Step 4 – Continuing with pagination

When the response provides a `continuation_token`, the agent can request the next page by reusing the token inside the reference frame. This keeps multi-page exchanges deterministic and prevents accidental duplication.

In [ ]:
next_page_request = MCPMessage(
    type="resource_request",
    frame=ReferenceFrame(
        session_id=resource_request.frame.session_id,
        namespace=resource_request.frame.namespace,
        continuation_token=resource_response.frame.continuation_token,
    ),
    payload={"resource": "vector_store", "query": "recent incidents involving authentication"},
    constraints={"limit": 5},
)

next_page_request.validate()
next_page_request.describe()


### Safety checklist

Before wiring MCP into production systems, run through this checklist:

1. **Namespace enforcement:** Confirm that every tool checks the namespace in incoming frames and rejects mismatches.
2. **Rate limiting:** Apply per-session rate limits to prevent abuse.
3. **Pagination guardrails:** Require `limit` values and cap them to prevent expensive queries.
4. **Audit logging:** Record the full JSON messages so that human operators can review what the agent requested and why.
5. **Error handling:** Always send an explicit `error` message type when a request fails, including a human-readable description.

### Exercise – Create an error response

An MCP tool should send an `error` message if it cannot complete a request. Imagine that the `vector_store` tool receives a request for an unknown namespace. Sketch the structure of the error message before revealing the sample answer below.

In [ ]:
error_response = MCPMessage(
    type="error",
    frame=make_reference_frame(namespace="project-unknown"),
    payload={
        "code": "namespace_not_found",
        "message": "The namespace 'project-unknown' is not registered for this tool.",
        "suggestion": "Verify the namespace or request access from the administrator.",
    },
)

error_response.validate()
error_response.describe()


## Putting it all together

The flow below summarizes the mini-workflow you just implemented:

1. **session_init** → agent introduces itself and negotiates capabilities.
2. **resource_request** → agent asks for specific data with guardrails.
3. **resource_response** → tool replies with structured items and optional continuation token.
4. **resource_request (next page)** → agent continues paging until finished.
5. **error** → tool uses explicit error messages to signal problems.

By structuring your traffic with MCP you gain reproducibility, observability, and safety—all of which are essential for production-grade AI agents.

## Next steps

- **Explore the official specification:** Search for the latest MCP reference documentation and compare the fields used there with the simplified version in this notebook.
- **Integrate with your own tools:** Adapt the dataclasses to match the actual schema your platform expects and plug them into your agent's tool-calling logic.
- **Add automated tests:** Use the `validate()` method from this notebook to build unit tests that reject malformed messages before they reach downstream services.
- **Keep iterating:** Protocols evolve. Revisit your implementation regularly to incorporate new safety recommendations and capabilities.

You are now ready to move on to more advanced topics such as the Agent-to-Agent protocol or streaming resource updates.